In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [14]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [15]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=1)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

            # update tree
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, rads_bow_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     61   10    9  111.45  491  110.45  1.00  0.01  103.91  459  102.65   
10000    52   20   19  111.07  473  109.80  1.25  0.01  103.68  443  102.20   
15000    51   30   29  110.86  463  109.44  1.41  0.01  103.68  439  102.03   
20000    51   40   39  110.72  456  109.19  1.52  0.01  103.60  433  101.83   
25000    51   50   49  110.60  450  108.97  1.62  0.01  103.38  424  101.47   
30000    50   60   59  110.51  446  108.81  1.69  0.01  103.32  420  101.35   
35000    55   70   69  110.44  442  108.67  1.76  0.01  103.32  418  101.25   
40000    51   80   79  110.38  439  108.56  1.81  0.01  103.28  418  101.23   
45000    56   90   89  110.32  437  108.45  1.86  0.01  103.19  413  101.06   
50000    53  100   99  110.27  434  108.37  1.90  0.01  103.15  412  101.02   
55000    60  110  109  110.23  432  108.29  1.94  0.01  103.09  411  100.91   
60000    49  120  119  110.20  431  108.24  1.96  0.01  103.36  418  101.29   
65000    54  130  129  110.17  430  108.18  1.98  0.01  103.09  410  100.93   
70000    52  140  139  110.14  429  108.13  2.01  0.00  103.09  411  100.95   
75000    54  150  149  110.12  427  108.09  2.03  0.00  103.08  410  100.90   
80000    53  160  159  110.10  426  108.05  2.04  0.00  103.06  410  100.89   
85000    55  170  169  110.07  425  108.01  2.06  0.00  103.10  409  100.88   
90000    54  180  179  110.06  425  107.97  2.08  0.00  102.99  407  100.81   
95000    55  190  189  110.04  424  107.94  2.09  0.00  103.11  410  100.87   
100000   57  200  199  110.02  423  107.91  2.11  0.00  103.05  408  100.81   
105000   55  210  209  110.01  422  107.88  2.12  0.00  103.09  409  100.84   
110000   53  220  219  109.99  422  107.86  2.13  0.00  103.06  408  100.84   
115000   61  230  229  109.98  421  107.83  2.14  0.00  103.06  406  100.76   
120000   53  240  239  109.97  420  107.81  2.15  0.00  103.15  410  100.90   
125000   59  250  249  109.95  420  107.78  2.17  0.00  103.02  405  100.72   
130000   51  260  259  109.94  419  107.76  2.17  0.00  103.00  405  100.75   
135000   58  270  269  109.93  419  107.74  2.18  0.00  102.93  402  100.62   
140000   51  280  279  109.92  418  107.72  2.19  0.00  102.91  403  100.66   
145000   59  290  289  109.91  418  107.70  2.20  0.00  102.92  402  100.61   
150000   52  300  299  109.90  417  107.68  2.21  0.00  102.95  404  100.69   
...     ...  ...  ...     ...  ...     ...   ...   ...     ...  ...     ...   
225000   59  450  449  109.80  412  107.49  2.30  0.00  102.88  400  100.54   
230000   54  460  459  109.79  412  107.48  2.30  0.00  102.93  403  100.63   
235000   60  470  469  109.79  412  107.47  2.31  0.00  102.91  401  100.55   
240000   53  480  479  109.78  412  107.47  2.31  0.00  102.97  404  100.69   
245000   59  490  489  109.78  412  107.46  2.31  0.00  102.89  401  100.55   
250000   52  501    0  109.77  411  107.45  2.32  0.00  102.92  402  100.62   
255000   58  511   10  109.77  411  107.44  2.32  0.00  102.92  401  100.57   
260000   50  521   20  109.76  411  107.43  2.32  0.00  102.95  403  100.66   
265000   58  531   30  109.76  411  107.42  2.33  0.00  102.93  401  100.58   
270000   50  541   40  109.76  411  107.42  2.33  0.00  102.99  404  100.68   
275000   57  551   50  109.75  411  107.41  2.33  0.00  102.96  401  100.58   
280000   51  561   60  109.75  410  107.41  2.34  0.00  103.10  406  100.78   
285000   57  571   70  109.75  410  107.40  2.34  0.00  103.01  402  100.63   
290000   50  581   80  109.74  410  107.39  2.34  0.00  103.03  404  100.73   
295000   57  591   90  109.74  410  107.38  2.35  0.00  102.95  401  100.59   
300000   51  601  100  109.74  410  107.38  2.35  0.00  102.97  402  100.65   
305000   58  611  110  109.73  410  107.37  2.35  0.00  102.89  399  100.52   
310000   49  621  120  109.73  409  107.37  2.36  0.00  102.93  401  100.

0 R: 1.000 P: 0.367 carry pockets ! room back strap work space lot nice
   1 R: 0.212 P: 0.104 pocket power inch mouse netbook small room charger cord tablet
     11 R: 0.038 P: 0.038 ; & pro size quality price big made bought air
     12 R: 0.070 P: 0.070 amazon item $ price ordered ... quality received return buy
   2 R: 0.154 P: 0.070 sleeve inside protection zipper padding pro soft neoprene snug protect
     23 R: 0.083 P: 0.083 ! love color mac perfectly perfect recommend smell pro cute
   4 R: 0.197 P: 0.066 protection protect - air scratches : nice bit pro price
     41 R: 0.070 P: 0.070 cover color keyboard apple pro hard mac love easy pink
     43 R: 0.060 P: 0.060 bottom top cover plastic feet back easily nice part speck
   3 R: 0.071 P: 0.031 zipper strap - : shoulder zippers open flap velcro side
     31 R: 0.040 P: 0.040 months broke years started year bought weeks ago month week
{0: [1, 2, 4, 5], 1: [12, 13], 2: [23, 21], 4: [41, 43, 42], 5: [51]}
